### Installazione delle dipendenze

Richiede Python3.10 o superiore. Dovresti creare un virtual environment con il comando:
```bash
python -m venv .venv
```

Dopodichè puoi eseguire la cella successiva dentro al virtual environment.

In [55]:
!python --version
!pip install -r requirements.txt

Python 3.10.4


You should consider upgrading via the 'C:\Users\tonto\OneDrive\Desktop\magistrale\bd2\projects\cap\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [56]:
import pandas as pd
import numpy as np

### Definizione costanti

In [76]:
DATA_PATH = "pg_stat_statements_16_20_60s.csv"
CALLS = [30, 29]

### Importazione statistiche da file CSV

In [77]:
df = pd.read_csv(DATA_PATH)
df.head()

,userid,dbid,toplevel,queryid,query,plans,total_plan_time,min_plan_time,max_plan_time,mean_plan_time,...,wal_fpi,wal_bytes,jit_functions,jit_generation_time,jit_inlining_count,jit_inlining_time,jit_optimization_count,jit_optimization_time,jit_emission_count,jit_emission_time
0,10,16419,True,-3825640688184167069,SELECT * FROM pg_stat_database,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,16419,True,1345865735253463468,SELECT * FROM pg_stat_user_tables,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10,16419,True,3545412574037053608,SET DateStyle=ISO,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10,16419,True,-6911109443227572258,SELECT * FROM pg_stat_bgwriter,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,16419,True,-141619989335985508,SELECT * FROM pg_stat_archiver,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
df.columns

Index(['userid', 'dbid', 'toplevel', 'queryid', 'query', 'plans',
       'total_plan_time', 'min_plan_time', 'max_plan_time', 'mean_plan_time',
       'stddev_plan_time', 'calls', 'total_exec_time', 'min_exec_time',
       'max_exec_time', 'mean_exec_time', 'stddev_exec_time', 'rows',
       'shared_blks_hit', 'shared_blks_read', 'shared_blks_dirtied',
       'shared_blks_written', 'local_blks_hit', 'local_blks_read',
       'local_blks_dirtied', 'local_blks_written', 'temp_blks_read',
       'temp_blks_written', 'blk_read_time', 'blk_write_time',
       'temp_blk_read_time', 'temp_blk_write_time', 'wal_records', 'wal_fpi',
       'wal_bytes', 'jit_functions', 'jit_generation_time',
       'jit_inlining_count', 'jit_inlining_time', 'jit_optimization_count',
       'jit_optimization_time', 'jit_emission_count', 'jit_emission_time'],
      dtype='object')

### Eliminazione delle colonne che non ci servono

In [78]:
columns = ["query", "calls", "total_exec_time", "min_exec_time", "max_exec_time", "mean_exec_time", "stddev_exec_time", "blk_read_time", "blk_write_time"]
df = df[columns]
df = df[df["calls"].isin(CALLS)]

### Calcolo delle variabili operazionali

Aggiustiamo il tempo di risposta rimuovendo i valori massimo e minimo, considerandoli outliers.

Calcoliamo il service demand di CPU e disco.

Infine convertiamo tutto in secondi.

In [79]:
df["blk_read_time"] /= 1000
df["blk_write_time"] /= 1000
df["total_exec_time"] /= 1000
df["min_exec_time"] /= 1000
df["max_exec_time"] /= 1000
df["mean_exec_time"] /= 1000

df["mean_blk_read_time"] = df["blk_read_time"] / df["calls"]
df["mean_blk_write_time"] = df["blk_write_time"] / df["calls"]
df["adj_mean_exec_time"] = (df["total_exec_time"] - df["min_exec_time"] - df["max_exec_time"]) / (df["calls"] - 2)
df["D_CPU"] = df["adj_mean_exec_time"] - df["mean_blk_read_time"] - df["mean_blk_write_time"]
df["D_DISK"] = df["mean_blk_read_time"] + df["mean_blk_write_time"]
df["CPU_vs_DISK"] = df["D_CPU"] / df["D_DISK"]

In [80]:
df

,query,calls,total_exec_time,min_exec_time,max_exec_time,mean_exec_time,stddev_exec_time,blk_read_time,blk_write_time,mean_blk_read_time,mean_blk_write_time,adj_mean_exec_time,D_CPU,D_DISK,CPU_vs_DISK
6,"SELECT\n p_brand,\n p_type,\n p_size,\n ...",30,26.965064,0.862054,1.401235,0.898835,94.622963,0.430790,0.0,0.014360,0.0,0.882206,0.867847,0.014360,60.436471
22,"select s_name, s_address from supplier, nation...",29,35.798148,0.999820,2.774756,1.234419,317.583333,23.278985,0.0,0.802724,0.0,1.186058,0.383335,0.802724,0.477542


### Query CPU-Intensive

In [86]:
q_cpu = df.loc[df["CPU_vs_DISK"].idxmax()]
print(q_cpu)
print()
print(q_cpu["query"])

query                  SELECT\n   p_brand,\n   p_type,\n   p_size,\n ...
calls                                                                 30
total_exec_time                                                26.965064
min_exec_time                                                   0.862054
max_exec_time                                                   1.401235
mean_exec_time                                                  0.898835
stddev_exec_time                                               94.622963
blk_read_time                                                    0.43079
blk_write_time                                                       0.0
mean_blk_read_time                                               0.01436
mean_blk_write_time                                                  0.0
adj_mean_exec_time                                              0.882206
D_CPU                                                           0.867847
D_DISK                                             

### Query Disk-Intensive

In [85]:
q_disk = df.loc[df["CPU_vs_DISK"].idxmin()]
print(q_disk)
print()
print(q_disk["query"])

query                  select s_name, s_address from supplier, nation...
calls                                                                 29
total_exec_time                                                35.798148
min_exec_time                                                    0.99982
max_exec_time                                                   2.774756
mean_exec_time                                                  1.234419
stddev_exec_time                                              317.583333
blk_read_time                                                  23.278985
blk_write_time                                                       0.0
mean_blk_read_time                                              0.802724
mean_blk_write_time                                                  0.0
adj_mean_exec_time                                              1.186058
D_CPU                                                           0.383335
D_DISK                                             